# Download all EUR LD matrices on Sherlock

Each job will download 5000 files, which should take roughly 30 hours. Since there are 124281 files, we will submit 25 jobs.

In [ ]:
# submission script
function run_repeats()
    for start_from in 1:5000:125000
        # create .sh file to submit jobs
        filename = "submit.sh"
        mem = 10
        open(filename, "w") do io
            println(io, "#!/bin/bash")
            println(io, "#")
            println(io, "#SBATCH --job-name=pan-ukbb")
            println(io, "#")
            println(io, "#SBATCH --time=48:00:00")
            println(io, "#SBATCH --cpus-per-task=1")
            println(io, "#SBATCH --mem-per-cpu=$(mem)G")
            println(io, "#SBATCH --partition=normal,candes")
            println(io, "")
            println(io, "#save job info on joblog:")
            println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
            println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
            println(io, "")
            println(io, "# load the job environment:")
            println(io, "module load java/11.0.11")
            println(io, "module load python/3.9.0")
            println(io, "module load julia/1.6.2")
            println(io, "export JULIA_DEPOT_PATH=\"/scratch/users/bbchu/.julia\"")
            println(io, "")
            println(io, "# run code")
            println(io, "echo 'julia download_ukbb.jl start_from $start_from'")
            println(io, "julia download_ukbb.jl $start_from")
            println(io, "")
            println(io, "#echo job info on joblog:")
            println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
            println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
            println(io, "#echo \" \"")
        end
        # submit job
        run(`sbatch $filename`)
        rm(filename, force=true)
    end
end
run_repeats()

In [ ]:
# download script
# usage: julia download_ukbb.jl 1

using EasyLD

population = "EUR"
outdir = pwd()
start_from = parse(Int, ARGS[1])
num_files = 5000

download_ukb_LD_matrices(population, outdir, start_from=start_from, num_files=num_files)